# 부록 10.1: 프롬프트 체이닝

- [수업](#lesson)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 도우미 함수를 설정하세요.

In [ ]:
!pip install anthropic

# Python의 내장 정규 표현식 라이브러리 가져오기
import re
import anthropic

# IPython 저장소에서 API_KEY 및 MODEL_NAME 변수 검색
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# 임의 길이의 메시지 목록을 받도록 다시 작성됨
def get_completion(messages, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages
    )
    return message.content[0].text

---

## 수업

"글쓰기는 다시 쓰는 것"이라는 말이 있습니다. 놀랍게도, **Claude는 그렇게 하도록 요청받았을 때 종종 응답의 정확도를 향상시킬 수 있습니다**!

Claude에게 "다시 생각해보라"고 요청하는 방법은 여러 가지가 있습니다. 인간에게 자신의 작업을 다시 확인하도록 요청하는 자연스러운 방법은 일반적으로 Claude에게도 효과가 있습니다. (프롬프트 체이닝을 사용하는 시기와 방법에 대한 추가 예제는 [프롬프트 체이닝 문서](https://docs.anthropic.com/claude/docs/chain-prompts)를 확인하세요.)

### 예제

이 예제에서는 Claude에게 열 개의 단어를 생각해내도록 요청합니다... 하지만 그 중 하나 이상은 실제 단어가 아닙니다.

In [ ]:
# 초기 프롬프트
first_user = "정확히 'ab'로 끝나는 단어 열 개를 말해주세요."

# API 메시지 배열
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Claude의 응답 저장 및 출력
first_response = get_completion(messages)
print(first_response)

**Claude에게 답변을 더 정확하게 만들도록 요청하면** 오류가 수정됩니다!

아래에서는 위의 Claude의 잘못된 응답을 가져와서 이전 답변을 수정하도록 요청하는 대화에 다른 차례를 추가했습니다.

In [ ]:
second_user = "실제 단어가 아닌 모든 '단어'에 대한 대체품을 찾아주세요."

# API 메시지 배열
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Claude의 응답 출력
print("------------------------ 변수 대체가 있는 전체 메시지 배열 ------------------------")
print(messages)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(messages))

하지만 Claude가 우리가 그렇게 말했기 때문에 답변을 수정하는 것일까요? 이미 정확한 답변으로 시작하면 어떻게 될까요? Claude가 자신감을 잃을까요? 여기서는 `first_response` 자리에 정확한 응답을 배치하고 다시 확인하도록 요청했습니다.

In [ ]:
first_user = "정확히 'ab'로 끝나는 단어 열 개를 말해주세요."

first_response = """'ab'로 끝나는 10개의 단어는 다음과 같습니다:

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "실제 단어가 아닌 모든 '단어'에 대한 대체품을 찾아주세요."

# API 메시지 배열
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Claude의 응답 출력
print("------------------------ 변수 대체가 있는 전체 메시지 배열 ------------------------")
print(messages)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(messages))

위 블록에서 응답을 몇 번 생성하면 Claude가 대부분의 경우 단어를 그대로 두지만, 모두 이미 정확함에도 불구하고 여전히 가끔 단어를 변경한다는 것을 알 수 있습니다. 이를 완화하기 위해 무엇을 할 수 있을까요? 8장에 따르면, Claude에게 회피 방법을 제공할 수 있습니다! 한 번 더 시도해 보겠습니다.

In [ ]:
first_user = "정확히 'ab'로 끝나는 단어 열 개를 말해주세요."

first_response = """'ab'로 끝나는 10개의 단어는 다음과 같습니다:

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "실제 단어가 아닌 모든 '단어'에 대한 대체품을 찾아주세요. 모든 단어가 실제 단어라면 원래 목록을 반환하세요."

# API 메시지 배열
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Claude의 응답 출력
print("------------------------ 변수 대체가 있는 전체 메시지 배열 ------------------------")
print(messages)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(messages))

위 코드에서 응답을 몇 번 생성하여 Claude가 이제 자신의 입장을 고수하는 데 훨씬 더 나아졌는지 확인해보세요.

프롬프트 체이닝을 사용하여 **Claude에게 응답을 더 좋게 만들도록 요청**할 수도 있습니다. 아래에서는 Claude에게 먼저 이야기를 쓰고, 그 다음에 작성한 이야기를 개선하도록 요청했습니다. 개인적인 취향은 다를 수 있지만, 많은 사람들이 Claude의 두 번째 버전이 더 낫다는 데 동의할 것입니다.

먼저, Claude의 이야기 첫 번째 버전을 생성해 보겠습니다.

In [ ]:
# 초기 프롬프트
first_user = "달리기를 좋아하는 소녀에 관한 세 문장의 짧은 이야기를 작성하세요."

# API 메시지 배열
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Claude의 응답 저장 및 출력
first_response = get_completion(messages)
print(first_response)

이제 Claude에게 첫 번째 초안을 개선하도록 해보겠습니다.

In [ ]:
second_user = "이야기를 더 좋게 만들어주세요."

# API 메시지 배열
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Claude의 응답 출력
print("------------------------ 변수 대체가 있는 전체 메시지 배열 ------------------------")
print(messages)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(messages))

이러한 형태의 대체는 매우 강력합니다. 우리는 대체 자리 표시자를 사용하여 목록, 단어, Claude의 이전 응답 등을 전달했습니다. **"함수 호출"이라고 부르는 것을 수행하기 위해 대체를 사용할 수도 있습니다. 이는 Claude에게 어떤 함수를 수행하도록 요청한 다음, 그 함수의 결과를 가져와서 Claude에게 그 결과로 더 많은 작업을 수행하도록 요청하는 것입니다**. 이는 다른 대체와 마찬가지로 작동합니다. 이에 대한 자세한 내용은 다음 부록에서 다룹니다.

아래는 Claude에 대한 한 번의 호출 결과를 가져와 다른 더 긴 호출에 연결하는 또 다른 예입니다. 먼저 첫 번째 프롬프트로 시작해 보겠습니다(이번에는 Claude의 응답을 미리 채우기 포함).

In [ ]:
first_user = """아래 텍스트에서 모든 이름을 찾으세요:

"안녕, Jesse. 나 Erin이야. Joey가 내일 여는 파티에 대해 전화했어. Keisha가 온다고 했고 Mel도 올 것 같아."""

prefill = "<names>"

# API 메시지 배열
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Claude의 응답 저장 및 출력
first_response = get_completion(messages)
print("------------------------ 변수 대체가 있는 전체 메시지 배열 ------------------------")
print(messages)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(first_response)

이 이름 목록을 다른 프롬프트에 전달해 보겠습니다.

In [ ]:
second_user = "목록을 알파벳 순으로 정렬하세요."

# API 메시지 배열
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Claude의 응답 출력
print("------------------------ 변수 대체가 있는 전체 메시지 배열 ------------------------")
print(messages)
print("\n------------------------------------- Claude의 응답 -------------------------------------")
print(get_completion(messages))

이제 프롬프트 체이닝에 대해 배웠으니, 부록 10.2로 이동하여 프롬프트 체이닝을 사용하여 함수 호출을 구현하는 방법을 알아보세요.